<a href="https://colab.research.google.com/github/tinachung49/Portfolio/blob/main/EIS%20as%20A%20Variance%20Reduction%20Technique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Default Probability Estimation with Python Implementation and Performance Comparison

Convert Matlab Program 1-1 to Python code

In [ ]:
import numpy as np

# --- Basic Monte Carlo Method ---
N = 10000
c = 2

X = np.random.randn(N)
X1 = (X > c)
default_prob = np.mean(X1)
print(f"違約機率 (Default Prob): {default_prob}")

var_mc = np.var(X1, ddof=1)
print(f"變異數 (Variance): {var_mc}")

se_mc = np.sqrt(var_mc / N)
print(f"標準誤 (Standard Error): {se_mc}\n")

# --- (2) Efficient Importance Sampling (μ = c) ---
mu = c
Z = X + mu
Z1 = (Z > c) * np.exp(mu**2 * 0.5 - mu * Z)

is_prob = np.mean(Z1)
print(f"違約機率 (IS Estimate): {is_prob}")

var_is = np.var(Z1, ddof=1)
print(f"變異數 (Variance): {var_is}")

se_is = np.sqrt(var_is / N)
print(f"標準誤 (Standard Error): {se_is}")

違約機率 (Default Prob): 0.0227
變異數 (Variance): 0.022186928692869277
標準誤 (Standard Error): 0.0014895277336414142

違約機率 (IS Estimate): 0.02308491066219843
變異數 (Variance): 0.0012473009887415264
標準誤 (Standard Error): 0.0003531714864965073


Comparison with BMC and IS in CPU and GPU

In [ ]:
!pip install cupy-cuda-nvcc -q
import cupy as cp
print(f"CuPy 安裝成功！版本為: {cp.__version__}\n")
!nvidia-smi

ERROR: Could not find a version that satisfies the requirement cupy-cuda-nvcc (from versions: none)
ERROR: No matching distribution found for cupy-cuda-nvcc
CuPy 安裝成功！版本為: 13.3.0

Fri Oct 10 06:54:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P0             30W /   70W |     148M

In [ ]:
import time
import pandas as pd
import numpy as np
from scipy.stats import norm

try:
    import cupy as cp
    GPU_AVAILABLE = True
except ImportError:
    GPU_AVAILABLE = False
    print("警告：未找到 CuPy 函式庫或 CUDA 環境。將跳過 GPU 計算部分。")

def run_cpu_simulation(N, random_numbers):
    """使用 NumPy 在 CPU 上執行模擬，並回傳詳細結果"""
    results = []
    for c in range(1, 5):
        start_time = time.time()

        dp_true = norm.cdf(-c)
        X1 = (random_numbers > c)
        default_prob = np.mean(X1)
        var_mc = np.var(X1, ddof=1)
        se_mc = np.sqrt(var_mc / N)

        elapsed_time = time.time() - start_time

        results.append({
            'c': c,
            'DP_true': dp_true,
            'Mean': default_prob,
            'SE': se_mc,
            'time': elapsed_time
        })
    return pd.DataFrame(results)

def run_gpu_simulation(N, random_numbers_gpu):
    """使用 CuPy 在 GPU 上執行模擬，並回傳詳細結果"""
    if not GPU_AVAILABLE:
        return None

    results = []
    for c in range(1, 5):
        start_time = time.time()

        X1 = (random_numbers_gpu > c)
        default_prob = cp.mean(X1)
        var_mc = cp.var(X1, ddof=1)
        se_mc = cp.sqrt(var_mc / N)
        elapsed_time = time.time() - start_time

        results.append({
            'c': c,
            'time': elapsed_time
        })
    return pd.DataFrame(results)

def format_final_table(cpu_df, gpu_df):
    """將 CPU 和 GPU 的結果合併並格式化成最終表格"""

    final_table = pd.DataFrame()
    final_table['c'] = cpu_df['c']
    final_table['DP_true'] = cpu_df['DP_true'].apply(
        lambda x: f"{x:.4f}" if x > 1e-4 else f"{x:.2E}"
    )

    def combine_mean_se(row):
        if row['c'] < 4:
            return f"{row['Mean']:.4f} ({row['SE']:.4f})"
        else:
            if row['Mean'] == 0:
                return "-- (--)"
            else:
                return f"{row['Mean']:.2E} ({row['SE']:.2E})"

    final_table['BMC Mean (SE)'] = cpu_df.apply(combine_mean_se, axis=1)

    final_table['CPU Time (s)'] = cpu_df['time'].apply(lambda x: f"{x:.6f}")
    if gpu_df is not None:
        final_table['GPU Time (s)'] = gpu_df['time'].apply(lambda x: f"{x:.6f}")
    else:
        final_table['GPU Time (s)'] = "N/A"

    return final_table


if __name__ == "__main__":
    N = 1000000
    X = np.random.randn(N)
    if GPU_AVAILABLE:
        gpu_X = cp.asarray(X)
    else:
        gpu_X = None

    df_cpu_results = run_cpu_simulation(N, X)
    df_gpu_results = run_gpu_simulation(N, gpu_X)

    print("\n\n" + "="*70)
    print(" " * 15 + "Basic Monte Carlo 效能與結果比較")
    print("="*70)

    final_table = format_final_table(df_cpu_results, df_gpu_results)
    print(final_table.to_string(index=False))

    total_cpu_time = df_cpu_results['time'].sum()
    print(f"\nCPU 總時間: {total_cpu_time:.6f} 秒")

    if df_gpu_results is not None:
        total_gpu_time = df_gpu_results['time'].sum()
        speedup = total_cpu_time / total_gpu_time
        print(f"GPU 總時間: {total_gpu_time:.6f} 秒")
        print(f"GPU 約為 CPU 速度的 {speedup:.2f} 倍")



               Basic Monte Carlo 效能與結果比較
 c  DP_true       BMC Mean (SE) CPU Time (s) GPU Time (s)
 1   0.1587     0.1581 (0.0004)     0.007457     0.000662
 2   0.0228     0.0229 (0.0001)     0.004806     0.000381
 3   0.0013     0.0013 (0.0000)     0.004343     0.000347
 4 3.17E-05 3.20E-05 (5.66E-06)     0.004521     0.000306

CPU 總時間: 0.021127 秒
GPU 總時間: 0.001696 秒
GPU 約為 CPU 速度的 12.45 倍


In [ ]:
import time
import pandas as pd
import numpy as np
from scipy.stats import norm

try:
    import cupy as cp
    GPU_AVAILABLE = True
except ImportError:
    GPU_AVAILABLE = False
    print("警告：未找到 CuPy 函式庫或 CUDA 環境。將跳過 GPU 計算部分。")

def run_is_cpu_simulation(N, random_numbers):
    """使用 NumPy 在 CPU 上執行 IS 模擬，並回傳詳細結果"""
    results = []
    for c in range(1, 5):
        start_time = time.time()

        dp_true = norm.cdf(-c)
        mu = c
        Z = random_numbers + mu
        Z1 = (Z > c) * np.exp(mu**2 * 0.5 - mu * Z)
        is_prob = np.mean(Z1)
        var_is = np.var(Z1, ddof=1)
        se_is = np.sqrt(var_is / N)

        elapsed_time = time.time() - start_time

        results.append({
            'c': c,
            'DP_true': dp_true,
            'Mean': is_prob,
            'SE': se_is,
            'time': elapsed_time
        })
    return pd.DataFrame(results)

def run_is_gpu_simulation(N, random_numbers_gpu):
    """使用 CuPy 在 GPU 上執行 IS 模擬，並回傳詳細結果"""
    if not GPU_AVAILABLE:
        return None

    results = []
    for c in range(1, 5):
        start_time = time.time()

        mu = c
        Z = random_numbers_gpu + mu
        Z1 = (Z > c) * cp.exp(mu**2 * 0.5 - mu * Z)
        is_prob = cp.mean(Z1)
        var_is = cp.var(Z1, ddof=1)
        se_is = cp.sqrt(var_is / N)

        cp.cuda.Stream.null.synchronize()
        elapsed_time = time.time() - start_time

        results.append({
            'c': c,
            'time': elapsed_time
        })
    return pd.DataFrame(results)

def format_final_table(cpu_df, gpu_df):
    """將 CPU 和 GPU 的結果合併並格式化成 IS 最終表格"""

    final_table = pd.DataFrame()
    final_table['c'] = cpu_df['c']

    def format_value(value):
        if abs(value) < 1e-4 and value != 0:
            return f"{value:.2E}"
        else:
            return f"{value:.4f}"

    final_table['DP_true'] = cpu_df['DP_true'].apply(format_value)

    def combine_mean_se(row):
        mean_str = format_value(row['Mean'])
        se_str = format_value(row['SE'])
        return f"{mean_str} ({se_str})"

    final_table['IS Mean (SE)'] = cpu_df.apply(combine_mean_se, axis=1)
    final_table['CPU Time (s)'] = cpu_df['time'].apply(lambda x: f"{x:.6f}")
    if gpu_df is not None:
        final_table['GPU Time (s)'] = gpu_df['time'].apply(lambda x: f"{x:.6f}")
    else:
        final_table['GPU Time (s)'] = "N/A"

    return final_table


if __name__ == "__main__":
    N = 1000000
    X = np.random.randn(N)
    if GPU_AVAILABLE:
        gpu_X = cp.asarray(X)
    else:
        gpu_X = None

    df_cpu_results = run_is_cpu_simulation(N, X)
    df_gpu_results = run_is_gpu_simulation(N, gpu_X)

    print("\n\n" + "="*75)
    print(" " * 15 + "Importance Sampling (IS) 效能與結果比較")
    print("="*75)

    final_table = format_final_table(df_cpu_results, df_gpu_results)
    print(final_table.to_string(index=False))

    total_cpu_time = df_cpu_results['time'].sum()
    print(f"\nCPU 總時間: {total_cpu_time:.6f} 秒")

    if df_gpu_results is not None:
        total_gpu_time = df_gpu_results['time'].sum()
        speedup = total_cpu_time / total_gpu_time
        print(f"GPU 總時間: {total_gpu_time:.6f} 秒")
        print(f"GPU 約為 CPU 速度的 {speedup:.2f} 倍")



               Importance Sampling (IS) 效能與結果比較
 c  DP_true        IS Mean (SE) CPU Time (s) GPU Time (s)
 1   0.1587     0.1587 (0.0002)     0.018306     0.003056
 2   0.0228   0.0228 (3.49E-05)     0.016448     0.002935
 3   0.0013   0.0014 (2.49E-06)     0.026381     0.002864
 4 3.17E-05 3.17E-05 (6.75E-08)     0.026943     0.002847

CPU 總時間: 0.088078 秒
GPU 總時間: 0.011702 秒
GPU 約為 CPU 速度的 7.53 倍


#  C-VaR Estimation and Importance Sampling Algorithm Details

Compare BMC and IS methods for estimated means and standard er
rors, reproducing Table 1.2

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm

def cvar_basic_monte_carlo(c, N=1000000):
    """使用基本蒙地卡羅方法估計 C-VaR """
    X = np.random.randn(N)
    tail_samples = X[X > c]

    if len(tail_samples) < 2:
        return np.nan, np.nan

    cvar_mean = np.mean(tail_samples)
    cvar_se = np.std(tail_samples, ddof=1) / np.sqrt(len(tail_samples))

    return cvar_mean, cvar_se

def cvar_importance_sampling_final_corrected(c, N=1000000):
    """使用重要性抽樣方法估計 C-VaR"""
    Z = np.random.randn(N) + c
    weights = np.exp(c**2 * 0.5 - c * Z)

    numerator = np.sum(Z[Z > c] * weights[Z > c])
    denominator = np.sum(weights[Z > c])

    if denominator == 0:
        return np.nan, np.nan

    cvar_mean = numerator / denominator

    A_terms = Z * (Z > c) * weights
    B_terms = (Z > c) * weights

    var_ratio_terms = np.var(A_terms - cvar_mean * B_terms, ddof=1)
    mean_B = np.mean(B_terms)

    if mean_B == 0:
        return cvar_mean, np.nan

    cvar_var = var_ratio_terms / (N * mean_B**2)
    cvar_se = np.sqrt(cvar_var)

    return cvar_mean, cvar_se

def generate_comparison_table(c_values, N):
    """生成並格式化最終的比較表格"""
    results = []

    for c in c_values:
        bmc_mean, bmc_se = cvar_basic_monte_carlo(c, N)
        is_mean, is_se = cvar_importance_sampling_final_corrected(c, N)
        true_cvar = norm.pdf(c) / (1 - norm.cdf(c))

        results.append({
            'c': c,
            'BMC Mean': bmc_mean, 'BMC SE': bmc_se,
            'True C-VaR': true_cvar,
            'IS Mean': is_mean, 'IS SE': is_se
        })


    df = pd.DataFrame(results)
    formatted_table = pd.DataFrame()
    formatted_table['c'] = df['c'].astype(str)

    def format_bmc(row):
        if pd.isna(row['BMC Mean']): return "— (—)"
        return f"{row['BMC Mean']:.3f} ({row['BMC SE']:.3f})"

    def format_is(row):
        if pd.isna(row['IS Mean']): return "— (—)"
        return f"{row['IS Mean']:.3f} ({row['IS SE']:.4f})"

    formatted_table['BMC C-VaR (SE)'] = df.apply(format_bmc, axis=1)
    formatted_table['True C-VaR'] = df['True C-VaR'].apply(lambda x: f"{x:.3f}")
    formatted_table['IS C-VaR (SE)'] = df.apply(format_is, axis=1)

    return formatted_table

if __name__ == "__main__":
    N_SAMPLES = 1000000
    C_VALUES = [3, 4, 5]

    final_table = generate_comparison_table(C_VALUES, N_SAMPLES)

    print("\n" + "="*60)
    print("       C-VaR Estimation Comparison (BMC vs. IS)")
    print("="*60)
    print(final_table.to_string(index=False))
    print("="*60)


       C-VaR Estimation Comparison (BMC vs. IS)
c BMC C-VaR (SE) True C-VaR  IS C-VaR (SE)
3  3.262 (0.007)      3.283 3.283 (0.0004)
4  4.177 (0.025)      4.226 4.226 (0.0004)
5          — (—)      5.187 5.186 (0.0003)


# Extending Importance Sampling to E[XI(X > c)] Estimation

In [ ]:
import time
import pandas as pd
import numpy as np
from scipy.stats import norm

# 嘗試導入 cupy
try:
    import cupy as cp
    GPU_AVAILABLE = True
except ImportError:
    GPU_AVAILABLE = False

def estimate_bmc(c, N, lib=np):
    """ BMC 估計 E[X * I(X > c)]"""
    samples = lib.random.randn(N)
    tail_samples = samples[samples > c]

    if len(tail_samples) < 2:
        return lib.nan, lib.nan

    estimator_mean = lib.sum(tail_samples) / N
    g_squared_mean = lib.sum(tail_samples**2) / N
    estimator_var = (g_squared_mean - estimator_mean**2) / N
    estimator_se = lib.sqrt(estimator_var)

    return estimator_mean, estimator_se

def estimate_is_efficient(c, N, lib=np):
    """ IS 估計 E[X * I(X > c)]"""
    z_samples = lib.random.randn(N) + c
    weights = lib.exp(-c * z_samples + 0.5 * c**2)
    mask = z_samples > c

    estimator_terms = z_samples * mask * weights
    estimator_mean = lib.mean(estimator_terms)
    estimator_var = lib.var(estimator_terms, ddof=1)
    estimator_se = lib.sqrt(estimator_var / N)

    return estimator_mean, estimator_se

def run_all_simulations(c_values, N):
    """執行所有 CPU 和 GPU 模擬並返回原始數據 DataFrame"""
    results = []
    for c in c_values:
        row = {'c': c}
        row['True Value'] = norm.pdf(c)

        # --- CPU ---
        start_time = time.time()
        mean, se = estimate_bmc(c, N, lib=np)
        row['BMC_CPU_Time'] = time.time() - start_time
        row['BMC_CPU_Mean'], row['BMC_CPU_SE'] = mean, se

        start_time = time.time()
        mean, se = estimate_is_efficient(c, N, lib=np)
        row['IS_CPU_Time'] = time.time() - start_time
        row['IS_CPU_Mean'], row['IS_CPU_SE'] = mean, se

        # --- GPU ---
        if GPU_AVAILABLE:
            start_time = time.time()
            mean_gpu, se_gpu = estimate_bmc(c, N, lib=cp)
            row['BMC_GPU_Time'] = time.time() - start_time

            start_time = time.time()
            mean_gpu, se_gpu = estimate_is_efficient(c, N, lib=cp)
            row['IS_GPU_Time'] = time.time() - start_time
        else:
            row['BMC_GPU_Time'], row['IS_GPU_Time'] = np.nan, np.nan

        results.append(row)

    return pd.DataFrame(results)

def format_final_table_new(df):
    table = pd.DataFrame()
    table['c'] = df['c']

    table['True DP'] = df['True Value'].apply(lambda x: f"{x:.2E}")

    def format_bmc_mean_se(row):
        if pd.isna(row['BMC_CPU_Mean']):
            return "--- (NaN)"
        mean_str = f"{row['BMC_CPU_Mean']:.2E}"
        se_str = f"{row['BMC_CPU_SE']:.2E}"
        return f"{mean_str} ({se_str})"
    table['BMC Mean (SE)'] = df.apply(format_bmc_mean_se, axis=1)


    def format_is_mean_se(row):
        mean_str = f"{row['IS_CPU_Mean']:.2E}"
        se_str = f"{row['IS_CPU_SE']:.2E}"
        return f"{mean_str} ({se_str})"
    table['EIS Mean (SE)'] = df.apply(format_is_mean_se, axis=1)

    def format_cpu_time(row):
        bmc_time = row['BMC_CPU_Time']
        is_time = row['IS_CPU_Time']
        return f"BMC: {bmc_time:.3f} / EIS: {is_time:.3f}"
    table['CPU Time (s)'] = df.apply(format_cpu_time, axis=1)

    def format_gpu_time(row):
        if pd.isna(row['BMC_GPU_Time']):
            return "N/A"
        bmc_time = row['BMC_GPU_Time']
        is_time = row['IS_GPU_Time']
        return f"BMC: {bmc_time:.3f} / EIS: {is_time:.3f}"
    table['GPU Time (s)'] = df.apply(format_gpu_time, axis=1)

    return table

# --- 主執行程式 ---
if __name__ == "__main__":
    N_SAMPLES = 1000000
    C_VALUES = [3, 4, 5]

    results_df = run_all_simulations(C_VALUES, N_SAMPLES)

    final_table = format_final_table_new(results_df)

    print("\n" + "="*110)
    print(" " * 25 + "Performance Comparison for E[X * I(X > c)] Estimation")
    print("="*110)
    print(final_table.to_string(index=False))
    print("="*110)


                         Performance Comparison for E[X * I(X > c)] Estimation
 c  True DP       BMC Mean (SE)       EIS Mean (SE)            CPU Time (s)            GPU Time (s)
 3 4.43E-03 4.41E-03 (1.21E-04) 4.45E-03 (7.78E-06) BMC: 0.028 / EIS: 0.044 BMC: 0.002 / EIS: 0.001
 4 1.34E-04 1.31E-04 (2.35E-05) 1.34E-04 (2.76E-07) BMC: 0.026 / EIS: 0.043 BMC: 0.002 / EIS: 0.001
 5 1.49E-06           --- (NaN) 1.48E-06 (3.47E-09) BMC: 0.026 / EIS: 0.043 BMC: 0.002 / EIS: 0.001


# CPU VS GPU first version

CPU

In [ ]:
#BMC
import numpy as np
import pandas as pd
import time
from scipy.stats import norm

N = 1000000
X = np.random.randn(N)
results = []
total_time = 0

for c in range(1,5):
    start_time = time.time()
    dp_true = norm.cdf(-c)

    X1 = (X > c)
    default_prob = np.mean(X1)

    var_mc = np.var(X1, ddof=1)
    se_mc = np.sqrt(var_mc / N)

    elapsed_time = time.time() - start_time
    total_time += elapsed_time

    results.append({
        'c': c,
        'DP_true': dp_true,
        'Mean': default_prob,
        'SE': se_mc
    })
    print(f"c = {c} 耗時: {elapsed_time:.4f} 秒")

df = pd.DataFrame(results)
df_formatted = pd.DataFrame()
df_formatted['c'] = df['c']
df_formatted['DP_true'] = df['DP_true'].apply(
    lambda x: f"{x:.4f}" if x > 1e-4 else f"{x:.2E}"
)

if df.loc[df['c'] == 4, 'Mean'].iloc[0] == 0:
    mean_c4_str = '--'
    se_c4_str = '--'
else:
    mean_c4_str = f"{df.loc[df['c'] == 4, 'Mean'].iloc[0]:.2E}"
    se_c4_str = f"{df.loc[df['c'] == 4, 'SE'].iloc[0]:.2E}"

df_formatted['Mean'] = [f"{mean:.4f}" for mean in df['Mean'][:3]] + [mean_c4_str]
df_formatted['SE'] = [f"{se:.4f}" for se in df['SE'][:3]] + [se_c4_str]


print("\n--- Basic MC 結果")
df_formatted.rename(columns={'Mean': 'MC Mean', 'SE': 'MC SE'}, inplace=True)
print(df_formatted.to_string(index=False))
print(f"\ntotal time: {total_time:.4f} 秒")

c = 1 耗時: 0.0128 秒
c = 2 耗時: 0.0074 秒
c = 3 耗時: 0.0069 秒
c = 4 耗時: 0.0074 秒

--- Basic MC 結果
 c  DP_true  MC Mean    MC SE
 1   0.1587   0.1585   0.0004
 2   0.0228   0.0225   0.0001
 3   0.0013   0.0013   0.0000
 4 3.17E-05 2.90E-05 5.39E-06

total time: 0.0346 秒


In [ ]:
#IS
import numpy as np
import pandas as pd
import time
from scipy.stats import norm

N = 1000000
X = np.random.randn(N)
results = []
total_time = 0

for c in range(1,5):
    start_time = time.time()
    dp_true = norm.cdf(-c)
    mu = c
    Z = X + mu
    Z1 = (Z > c) * np.exp(mu**2 * 0.5 - mu * Z)
    is_prob = np.mean(Z1)
    var_is = np.var(Z1, ddof=1)
    se_is = np.sqrt(var_is / N)

    elapsed_time = time.time() - start_time
    total_time += elapsed_time

    results.append({
        'c': c,
        'DP_true': dp_true,
        'Mean': is_prob,
        'SE': se_is
    })
    print(f"c = {c} 耗時: {elapsed_time:.4f} 秒")


df = pd.DataFrame(results)
def format_value(value):
    if value == 0:
        return "0.0000"
    # 對於絕對值小於 0.0001 的數值，使用科學記號
    elif abs(value) < 1e-4:
        return f"{value:.2E}"
    # 其他則使用標準小數格式
    else:
        return f"{value:.4f}"

df_formatted = pd.DataFrame()
df_formatted['c'] = df['c']
df_formatted['DP_true'] = df['DP_true'].apply(format_value)
df_formatted['Mean'] = df['Mean'].apply(format_value)
df_formatted['SE'] = df['SE'].apply(format_value)


print("\n--- IS(μ=c) 結果")

df_formatted.rename(columns={'Mean': 'Mean', 'SE': 'SE'}, inplace=True)
print(df_formatted.to_string(index=False))
print(f"\n總計算時間: {total_time:.4f} 秒")

c = 1 耗時: 0.0284 秒
c = 2 耗時: 0.0238 秒
c = 3 耗時: 0.0242 秒
c = 4 耗時: 0.0214 秒

--- IS(μ=c) 結果
 c  DP_true     Mean       SE
 1   0.1587   0.1589   0.0002
 2   0.0228   0.0228 3.48E-05
 3   0.0013   0.0014 2.48E-06
 4 3.17E-05 3.17E-05 6.73E-08

總計算時間: 0.0979 秒


GPU

In [ ]:
#BMC
import cupy as cp
import pandas as pd
import time
from scipy.stats import norm

N = 1000000
X = cp.random.randn(N)
results = []
total_time = 0

for c in range(1,5):
    start_time = time.time()
    dp_true = norm.cdf(-c)

    X1 = (X > c)
    default_prob = cp.mean(X1)
    var_mc = cp.var(X1, ddof=1)
    se_mc = cp.sqrt(var_mc / N)

    elapsed_time = time.time() - start_time
    total_time += elapsed_time

    results.append({
        'c': c,
        'DP_true': dp_true,
        'Mean': default_prob.get(),
        'SE': se_mc.get()
    })
    print(f"c = {c} 耗時: {elapsed_time:.4f} 秒")

df = pd.DataFrame(results)
df_formatted = pd.DataFrame()
df_formatted['c'] = df['c']
df_formatted['DP_true'] = df['DP_true'].apply(
    lambda x: f"{x:.4f}" if x > 1e-4 else f"{x:.2E}"
)

if df.loc[df['c'] == 4, 'Mean'].iloc[0] == 0:
    mean_c4_str = '--'
    se_c4_str = '--'
else:
    mean_c4_str = f"{df.loc[df['c'] == 4, 'Mean'].iloc[0]:.2E}"
    se_c4_str = f"{df.loc[df['c'] == 4, 'SE'].iloc[0]:.2E}"

df_formatted['Mean'] = [f"{mean:.4f}" for mean in df['Mean'][:3]] + [mean_c4_str]
df_formatted['SE'] = [f"{se:.4f}" for se in df['SE'][:3]] + [se_c4_str]


print("\n--- Basic MC 結果")
df_formatted.rename(columns={'Mean': 'MC Mean', 'SE': 'MC SE'}, inplace=True)
print(df_formatted.to_string(index=False))
print(f"\ntotal time: {total_time:.4f} 秒")

c = 1 耗時: 0.0013 秒
c = 2 耗時: 0.0010 秒
c = 3 耗時: 0.0010 秒
c = 4 耗時: 0.0009 秒

--- Basic MC 結果
 c  DP_true  MC Mean    MC SE
 1   0.1587   0.1588   0.0004
 2   0.0228   0.0230   0.0001
 3   0.0013   0.0014   0.0000
 4 3.17E-05 3.50E-05 5.92E-06

total time: 0.0042 秒


In [ ]:
#IS
import cupy as cp
import pandas as pd
import time
from scipy.stats import norm

N = 1000000
X = cp.random.randn(N)
results = []
total_time = 0

for c in range(1,5):
    start_time = time.time()
    dp_true = norm.cdf(-c)
    mu = c
    Z = X + mu
    Z1 = (Z > c) * cp.exp(mu**2 * 0.5 - mu * Z)
    is_prob = cp.mean(Z1)
    var_is = cp.var(Z1, ddof=1)
    se_is = cp.sqrt(var_is / N)

    elapsed_time = time.time() - start_time
    total_time += elapsed_time

    results.append({
        'c': c,
        'DP_true': dp_true,
        'Mean': is_prob.get(),
        'SE': se_is.get()
    })
    print(f"c = {c} 耗時: {elapsed_time:.4f} 秒")


df = pd.DataFrame(results)
def format_value(value):
    if value == 0:
        return "0.0000"
    # 對於絕對值小於 0.0001 的數值，使用科學記號
    elif abs(value) < 1e-4:
        return f"{value:.2E}"
    # 其他則使用標準小數格式
    else:
        return f"{value:.4f}"

df_formatted = pd.DataFrame()
df_formatted['c'] = df['c']
df_formatted['DP_true'] = df['DP_true'].apply(format_value)
df_formatted['Mean'] = df['Mean'].apply(format_value)
df_formatted['SE'] = df['SE'].apply(format_value)


print("\n--- IS(μ=c) 結果")

df_formatted.rename(columns={'Mean': 'Mean', 'SE': 'SE'}, inplace=True)
print(df_formatted.to_string(index=False))
print(f"\n總計算時間: {total_time:.4f} 秒")

c = 1 耗時: 0.0027 秒
c = 2 耗時: 0.0013 秒
c = 3 耗時: 0.0008 秒
c = 4 耗時: 0.0008 秒

--- IS(μ=c) 結果
 c  DP_true     Mean       SE
 1   0.1587   0.1585   0.0002
 2   0.0228   0.0227 3.48E-05
 3   0.0013   0.0013 2.48E-06
 4 3.17E-05 3.16E-05 6.72E-08

總計算時間: 0.0056 秒


In [ ]:
import cupy as cp
import time
N = 10000
c = 2

# --- (1) Basic Monte Carlo Method on GPU ---
start_time_mc = time.time()

X = cp.random.randn(N)
X1 = (X > c)
default_prob = cp.mean(X1)
var_mc = cp.var(X1, ddof=1)
se_mc = cp.sqrt(var_mc / N)

end_time_mc = time.time()

print(f"違約機率 (Default Prob): {default_prob.get()}")
print(f"變異數 (Variance): {var_mc.get()}")
print(f"標準誤 (Standard Error): {se_mc.get()}")
print(f"GPU 計算耗時: {end_time_mc - start_time_mc:.6f} 秒\n")

# --- (2) Efficient Importance Sampling on GPU ---
start_time_is = time.time()

mu = c
Z = X + mu
Z1 = (Z > c) * cp.exp(mu**2 * 0.5 - mu * Z)
is_prob = cp.mean(Z1)
var_is = cp.var(Z1, ddof=1)
se_is = cp.sqrt(var_is / N)

end_time_is = time.time()

print(f"違約機率 (IS Estimate): {is_prob.get()}")
print(f"變異數 (Variance): {var_is.get()}")
print(f"標準誤 (Standard Error): {se_is.get()}")
print(f"GPU 計算耗時: {end_time_is - start_time_is:.6f} 秒")

違約機率 (Default Prob): 0.0268
變異數 (Variance): 0.026084368436843683
標準誤 (Standard Error): 0.0016150655849482918
GPU 計算耗時: 2.523694 秒

違約機率 (IS Estimate): 0.022487583768284348
變異數 (Variance): 0.001184668628010523
標準誤 (Standard Error): 0.0003441901550030917
GPU 計算耗時: 0.747434 秒
